# simple_training.ipynb

Simple training implementation.

Author: Connacher Murphy

In [1]:
# Libraries
import pest_classification as pest

import numpy as np
import os
from sklearn.metrics import accuracy_score
import timm
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from types import SimpleNamespace

/home/connachermurphy/.local/share/virtualenvs/pest-classification-3JmtqzHf/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Split df into test and training
df = pest.df
test_df = df[df["set"] == "test_set"]
train_df = df[df["set"] == "train_set"]

In [3]:
# Dataloader
config = SimpleNamespace(**{})

config.batch_size = 32

config.image_dir = os.path.expanduser("~/data/ccmt/CCMT Dataset-Augmented")
config.image_size = 256

train_dataset = pest.AugmentedCCMT(config, train_df)

train_dataloader = DataLoader(
    train_dataset,
    batch_size=config.batch_size,
    shuffle=True,
    num_workers=0,
)

In [4]:
config.num_classes = 2
config.backbone = "resnet18"

model = timm.create_model(config.backbone, pretrained=True, num_classes=config.num_classes)

In [5]:
X = torch.randn(config.batch_size, 3, config.image_size, config.image_size)
y = model(X)
print(y.shape)
print(y)

torch.Size([32, 2])
tensor([[ 0.1885,  0.0836],
        [ 0.1114, -0.1112],
        [ 0.1331, -0.1206],
        [ 0.0235, -0.0378],
        [ 0.1708,  0.0206],
        [ 0.0974, -0.0513],
        [ 0.1918, -0.1082],
        [ 0.0855,  0.0058],
        [ 0.0763, -0.0084],
        [ 0.1577, -0.0373],
        [-0.0134, -0.0805],
        [ 0.0147, -0.1591],
        [ 0.1338, -0.1618],
        [ 0.1163, -0.0992],
        [ 0.1209,  0.0740],
        [ 0.1184, -0.1952],
        [ 0.0310,  0.0178],
        [-0.0087, -0.1515],
        [ 0.0395, -0.0114],
        [ 0.0075,  0.0390],
        [ 0.0196, -0.1012],
        [ 0.1361, -0.0273],
        [ 0.0822, -0.0033],
        [ 0.3051,  0.0308],
        [-0.0761, -0.0743],
        [ 0.1422, -0.1737],
        [ 0.0059, -0.0868],
        [ 0.1244, -0.0838],
        [ 0.1241,  0.1128],
        [ 0.2374, -0.0477],
        [ 0.0681,  0.0084],
        [ 0.0992, -0.0552]], grad_fn=<AddmmBackward0>)


In [6]:
# Hyperparameters
config.criterion = nn.CrossEntropyLoss()
config.lr = 1e-4
optimizer = torch.optim.Adam(model.parameters(), lr=config.lr, weight_decay=0.0)

In [7]:
# Use GPU if available
config.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.is_available()

False

In [8]:
model.to(config.device)

pest.train_epoch(train_dataloader, model, optimizer, config)

100%|██████████| 608/608 [26:52<00:00,  2.65s/it]


0.046059526902880815